This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
# API_KEY = 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import statistics

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2019-12-03&end_date=2019-12-03&api_key=' + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2019-11-20) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2019-12-04T22:18:40.512Z', 'newest_available_date': '2019-12-04', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2019-12-03', 'end_date': '2019-12-03', 'data': [['2019-12-03', 107.7, 110.1, 107.3, 108.9, None, 130237.0, 14162319.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, 
# for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

AFX_X_2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)

In [6]:
# 2. Convert the returned JSON object into a Python dictionary.

AFX_X_2017 = AFX_X_2017.json()
print('AFX_X_2017 is now a Python ' + str(type(AFX_X_2017)))

AFX_X_2017 is now a Python <class 'dict'>


In [7]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

# Gather the relevant data into a more useful format, a list of lists that can be indexed by column_name values.
data_keys = AFX_X_2017['dataset']['column_names']
data_list = AFX_X_2017['dataset']['data']

#disregard--------------------------------
#data_list_of_dicts = []
#data_dict = defaultdict(dict)
# Populate the 2 level dictionary with nested for loops. The outer key is the date and 
# inner keys are the remaining column_names.
#for entry in data_list:
#    for key in data_keys[1:]:
#        data_dict[entry[0]][key] = data_list[data_list.index(entry)][data_keys.index(key)]
#disregard--------------------------------

# Create a list of open prices for the dates in the list. Only append recorded values, skip if 'None'.
open_list = []
for i in range(len(data_list)):
    if type(data_list[i][data_keys.index('Open')]) == float:
        open_list.append(data_list[i][data_keys.index('Open')])

open_max = max(open_list)
open_min = min(open_list)

# Print solution. Note: Frankfurt Stock Exchange trades stock in Euros (€).

print('The highest opening price for the AFX_X stock in 2017 was €' + str(open_max) + ' (based on ' + str(len(open_list)) + ' days of data).')
print('The lowest opening price for the AFX_X stock in 2017 was €' + str(open_min) + ' (based on ' + str(len(open_list)) + ' days of data).')

The highest opening price for the AFX_X stock in 2017 was €53.11 (based on 252 days of data).
The lowest opening price for the AFX_X stock in 2017 was €34.0 (based on 252 days of data).


In [8]:
# 4. What was the largest change in any one day (based on High and Low price)?

# Create a list of (High - Low) price changes for the dates in the list. Only append recorded values, skip if 'None'.
changes_HL_list = []
for i in range(len(data_list)):    
    if (type(data_list[i][data_keys.index('High')]) == float) & (type(data_list[i][data_keys.index('Low')]) == float):
        changes_HL_list.append(data_list[i][data_keys.index('High')] - data_list[i][data_keys.index('Low')])

# Calculate and print maximum.
changes_HL_max = max(changes_HL_list)
print('The largest High/Low price change in any one day for the AFX_X stock in 2017 was €' + str(round(changes_HL_max,2)) + ' (based on ' + str(len(changes_HL_list)) + ' days of data).')

The largest High/Low price change in any one day for the AFX_X stock in 2017 was €2.81 (based on 255 days of data).


In [9]:
#5. What was the largest change between any two days (based on Closing Price)?

# Create a list of (close - open) price changes for the dates in the list. Only append recorded values, skip if 'None'.
changes_CO_list = []
for i in range(len(data_list)):    
    if (type(data_list[i][data_keys.index('Close')]) == float) & (type(data_list[i][data_keys.index('Open')]) == float):
        changes_CO_list.append(data_list[i][data_keys.index('Close')] - data_list[i][data_keys.index('Open')])

# Calculate and print maximum.        
changes_CO_max = max(changes_CO_list)
print('The largest price change between any two days for the AFX_X stock in 2017 was €' + str(round(changes_CO_max,2)) + ' (based on ' + str(len(changes_CO_list)) + ' days of data).')


The largest price change between any two days for the AFX_X stock in 2017 was €1.64 (based on 252 days of data).


In [10]:
#6. What was the average daily trading volume during this year?

# Create a list of traded volumes for the dates in the list. Only append recorded values, skip if 'None'.
volume_list = []
for i in range(len(data_list)):
    if type(data_list[i][data_keys.index('Traded Volume')]) == float:
        volume_list.append(data_list[i][data_keys.index('Traded Volume')])

# Calculate and print average.         
volume_mean = statistics.mean(volume_list)
print('The average daily trading volume for the AFX_X stock in 2017 was ' + str(round(volume_mean,2)) + ' shares (based on ' + str(len(volume_list)) + ' days of data).')

The average daily trading volume for the AFX_X stock in 2017 was 89124.34 shares (based on 255 days of data).


In [11]:
# 7. (Optional) What was the median trading volume during this year.

# Calculate and print average.         
volume_median = statistics.median(volume_list)
print('The median daily trading volume for the AFX_X stock in 2017 was ' + str(round(volume_median,2)) + ' shares (based on ' + str(len(volume_list)) + ' days of data).')

The median daily trading volume for the AFX_X stock in 2017 was 76286.0 shares (based on 255 days of data).
